In [1]:
!pip install tensorflow-gpu==2.1.0.rc1

     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 450kB 47.0MB/s 
     |████████████████████████████████| 3.8MB 55.2MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D

from tensorflow.keras.layers import Dense, Flatten, Conv2D,MaxPool2D,Dropout,BatchNormalization
from tensorflow.keras import Model,datasets


In [0]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (train_images, train_labels)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)

In [9]:
for step, (x,y) in enumerate(train_ds):
  imgk=x
  break;
imgk.shape

TensorShape([32, 32, 32, 3])

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    #self.batch1=BatchNormalization()
    #self.drop1=Dropout(0.5)
    self.pool1=MaxPool2D(2,2)
    self.conv2 = Conv2D(64, 3, activation='relu')
    #self.batch2=BatchNormalization()
    #self.drop2=Dropout(0.5)
    self.pool2=MaxPool2D(2,2)
    
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    #self.drop3=Dropout(0.5)
    self.d2 = Dense(10)

  def call(self, x):
    x= tf.reshape(x, (-1, 32,32,3))
    x = self.conv1(x)
    #x= self.batch1(x)
    #x = self.drop1(x)
    x = self.pool1(x)
    x = self.conv2(x)
    #x = self.batch2(x)
    #x= self.drop2(x)
    x = self.pool2(x)
    x = self.flatten(x)
    x = self.d1(x)
    #x = self.drop3(x)
    return self.d2(x)

# Create an instance of the model
model2 = MyModel()
model2.build(input_shape=(None, 32,32,3))

In [60]:
model2.summary()

Model: "my_model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           multiple                  896       
_________________________________________________________________
max_pooling2d_14 (MaxPooling multiple                  0         
_________________________________________________________________
conv2d_16 (Conv2D)           multiple                  18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling multiple                  0         
_________________________________________________________________
flatten_8 (Flatten)          multiple                  0         
_________________________________________________________________
dense_16 (Dense)             multiple                  295040    
_________________________________________________________________
dense_17 (Dense)             multiple                  1

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam(0.001)

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model2(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model2.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model2.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model2(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions) 



In [64]:
EPOCHS = 10

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    #print(images[1].shape)
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 1.3943700790405273, Accuracy: 50.06999969482422, Test Loss: 1.1269104480743408, Test Accuracy: 60.28000259399414
Epoch 2, Loss: 1.0443590879440308, Accuracy: 63.215999603271484, Test Loss: 1.0061243772506714, Test Accuracy: 64.56999969482422
Epoch 3, Loss: 0.9028653502464294, Accuracy: 68.35600280761719, Test Loss: 0.9443655014038086, Test Accuracy: 66.94999694824219
Epoch 4, Loss: 0.8020988702774048, Accuracy: 72.02200317382812, Test Loss: 0.9549679756164551, Test Accuracy: 67.0099945068